In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lib.utils.base_funtion import build_dataloaders
from lib.config.cfg_loader import  env_setting
cfg = env_setting(cfg_name=None)
loader_train, loader_val = build_dataloaders(cfg)

In [17]:
import matplotlib.pyplot as plt
from lib.models.layer.patch_embed import PatchEmbed
from lib.models.layer.RMT import RetBlock,RelPos2d
import torch
rgb = None
modal = None

device = torch.device("cuda:1" )
embed_dim = 768
patch_embed = PatchEmbed(patch_size=16, in_chans=6, embed_dim=embed_dim,flatten=False).to(device)
relpos = RelPos2d(embed_dim=embed_dim,num_heads=12,initial_value=1,heads_range=3).to(device)
ret_block = RetBlock(retention='whole',embed_dim=embed_dim,num_heads=12,ffn_dim=96).to(device)
template_score = None 


for i,data in enumerate(loader_train,1):
    print(i)
    print(data.keys())
    # print(data['test_class'])
    # print(data['dataset'])
    print("template_images",data['template_images'].shape)
    print("search_images",data['search_images'].shape)
    
    template = data['template_images'][0].to(device)
    search = data['search_images'][0].to(device)
    template,template_patch_num = patch_embed(template)
    print("template",template.shape)
    template = template.permute(0,2,3,1) # B,C,H,W -> B,H,W,C
    print(template.shape)
    
    rel_pos = relpos((8, 8))
    
    template_score = ret_block(template,retention_rel_pos = rel_pos)
    # rgb = data['template_images'][0,:,:3].to(device)
    # modal = data['template_images'][0,:,3:].to(device)
    # rgb,rgb_patch_num = patch_embed(rgb)
    # modal,modal_patch_num = patch_embed(modal)
    # print("rgb",rgb.shape)
    # print("rgb_num_patches",rgb_patch_num)
    # print("modal",modal.shape)
    # print("modal_num_patches",modal_patch_num)
    break   

1
odict_keys(['template_images', 'template_anno', 'search_images', 'search_anno', 'dataset', 'test_class', 'valid'])
[None, None, None, None, None, 'train', None, None, 'train', None, None, 'train', 'train', None, 'train', None]
['LasHeR', 'LasHeR', 'LasHeR', 'LasHeR', 'visevent', 'DepthTrack', 'LasHeR', 'LasHeR', 'DepthTrack', 'visevent', 'visevent', 'DepthTrack', 'DepthTrack', 'visevent', 'DepthTrack', 'LasHeR']
template_images torch.Size([1, 16, 6, 128, 128])
search_images torch.Size([1, 16, 6, 320, 320])
template torch.Size([16, 768, 8, 8])
torch.Size([16, 8, 8, 768])


In [20]:
print(template_score.shape)
# print(template_score)

torch.Size([16, 8, 8, 768])
